In [7]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

In [8]:
def get_stock_map_load():
    data = pd.read_csv('C:/big18/final/with/almost/전종목시세.csv', encoding='cp949')
    data =data[['종목코드','종목명','상장주식수']]
    result_map = {row['종목코드']: {'종목명': row['종목명'], '상장주식수': row['상장주식수']} for index, row in data.iterrows()}
    return result_map

def get_stock_share(code):
    map = get_stock_map_load()
    stock_share = map[f'{code}']['상장주식수']
    return stock_share

def get_stock_name(code):
    map = get_stock_map_load()
    stock_name = map[f'{code}']['종목명']
    return stock_name

In [9]:
codes = ['000660'] # 전처리할 종목 코드

In [10]:
# 상장주식수로 거래량 스케일링
def divide_volumes(row,share):
    try:
        divisor = share
        row['power'] = row['매수량'] - row['매도량']
        row['power'] = row['power']*1000000 / divisor
    except KeyError:
        print(f"코드 {row['code']}를 찾지 못했습니다.")
        input()
    return row

In [11]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

# 결과를 저장할 리스트

for code in codes:
    stock_name = get_stock_name(code) # 해당 코드의 종목명저장
    stock_share = get_stock_share(code) # 해당 코드의 상장주식 수
    
    df = pd.read_csv(f'C:\\big18\\final\\with\\almost\\real_data\\{code}.csv',encoding='cp949')
    df = df[['code','종가','매수량','매도량']]

    df = divide_volumes(df, stock_share)
    
    # 데이터 로드
    df=df[['code','종가', 'power']]
    df.rename(columns={'종가' : 'close'}, inplace=True)

    # Unique 코드 및 카운트 계산
    counts = df['code'].value_counts()
    counts_dict = counts.to_dict()
    
    # 결과를 저장할 리스트
    results = []
    
    for key in tqdm(counts_dict.keys()):
        value = counts_dict[key]
        for i in range(value - 12):   # 예측 분 -1
            start_idx = counts_dict[key] - value + i
            end_idx = start_idx + 9  # 묶을 분 -1

            # 벡터화하여 연산
            vectorList = []
            # for col in ['close','power']:
            #     diff = (df[col].values[start_idx:end_idx] - df[col].values[start_idx + 1:end_idx + 1]) 
            #     vectorList.extend(round(d, 2) for d in diff)
            diff = (df['close'].values[start_idx:end_idx] - df['close'].values[start_idx + 1:end_idx + 1]) / df['close'].values[start_idx:end_idx] * 10000
            vectorList.extend(round(d, 2) for d in diff)
            diff = (df['power'].values[start_idx:end_idx] - df['power'].values[start_idx + 1:end_idx + 1])
            vectorList.extend(round(d, 2) for d in diff)

            openValue = df['close'][start_idx + 12] - df['close'][start_idx + 9]
            if openValue > 0:
                vectorList.append(2.0)
            elif openValue == 0:
                vectorList.append(1.0)
            else:
                vectorList.append(0.0)

            results.append(vectorList)
            
        print(f'{code} {stock_name} 처리 완료')

    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(results)

    # CSV 파일에 저장
    results_df.to_csv(f'C:\\big18\\final\\with\\almost\\data\\{code}전체데이터Vector.csv', index=False)

100%|██████████| 1/1 [00:24<00:00, 24.77s/it]

000660 SK하이닉스 처리 완료


In [12]:
# import pandas as pd
# import glob

# # CSV 파일들이 있는 디렉토리 경로
# path = './trainStockData/'  # 여기에 CSV 파일들이 있는 경로를 입력하세요
# all_files = glob.glob(path + "/*.csv")  # 모든 CSV 파일 목록 가져오기
# all_files
# # 모든 CSV 파일을 읽고 하나의 데이터프레임으로 합치기
# dataframes = []
# for filename in all_files:
#     df = pd.read_csv(filename)  # CSV 파일 읽기
#     dataframes.append(df)  # 데이터프레임 리스트에 추가

# # 모든 데이터프레임을 하나로 합치기
# combined_df = pd.concat(dataframes, ignore_index=True)

# # 결과를 새로운 CSV 파일로 저장
# combined_df.to_csv('combined_data.csv', index=False)  # 원하는 파일명으로 저장
# print('저장완료')
